# 問題
27の処理に加えて、テンプレートの値からMediaWikiマークアップを可能な限り除去し、国の基本情報を整形せよ。

In [2]:
import re
import html
import pandas as pd

# --- 入力読み込み ---
df = pd.read_json("Jawiki Country.json.gz", lines=True)
uk_text = df.query('title == "イギリス"')["text"].values[0]

# --- 基礎情報ブロック抽出 ---
block_pat = r'^\{\{\s*基礎情報[^\n]*.*?^\}\}\s*$'
blocks = re.findall(block_pat, uk_text, flags=re.DOTALL | re.MULTILINE)
if not blocks:
    raise ValueError("基礎情報ブロックが見つかりません")
block = blocks[0]

# --- フィールド抽出 ---
field_pat = r'^\|\s*(.+?)\s*=\s*(.*?)(?=\n\||\n\}\}|$)'

# --- 27: 強調マークアップの除去 ---
def remove_emphasis(text: str) -> str:
    patterns = [
        (r"'''''(.*?)'''''", r"\1"),
        (r"'''(.*?)'''",     r"\1"),
        (r"''(.*?)''",       r"\1"),
    ]
    changed = True
    while changed:
        old = text
        for pat, repl in patterns:
            text = re.sub(pat, repl, text, flags=re.DOTALL)
        changed = (text != old)
    return text.strip()

# --- 27: 内部リンクの平文化 ---
def remove_linkmark(text: str) -> str:
    patterns = [
        # メディア系: 説明があれば説明
        (r"\[\[(?:ファイル|File|画像):[^|\]]+(?:\|[^|\]]+)*\|([^|\]=][^|\]]*)\]\]", r"\1", 0),
        # 説明なし → ファイル名
        (r"\[\[(?:ファイル|File|画像):([^\]|]+)\]\]", r"\1", 0),
        # [[記事|表示]] → 表示
        (r"\[\[[^|\]]+\|([^\]]+)\]\]", r"\1", 0),
        # [[記事]] / [[記事#節]] → 記事
        (r"\[\[([^\]|#]+)(?:#[^\]]+)?\]\]", r"\1", 0),
        # インターウィキ（パイプ付き） → 表示
        (r"\[\[:[a-z0-9\-]+:[^|\]]+\|([^\]]+)\]\]", r"\1", re.I),
        # インターウィキ（パイプなし） → タイトル（#節は落とす）
        (r"\[\[:[a-z0-9\-]+:([^\]#|]+)(?:#[^\]]+)?\]\]", r"\1", re.I),
    ]
    changed = True
    while changed:
        old = text
        for pat, repl, flg in patterns:
            text = re.sub(pat, repl, text, flags=flg | re.DOTALL)
        changed = (text != old)
    return text.strip()

# --- 28:  ---
def clean_markup_28(text: str) -> str:
    patterns = [
        # --- テンプレ（中身を残す系の最小セット） ---
        (r"\{\{lang\|[a-z0-9\-]+\|([^}]+)\}\}", r"\1", re.I),     # {{lang|xx|...}} → ...
        (r"\{\{仮リンク\|([^|}]+)\|[^}]+\}\}", r"\1", 0),        # {{仮リンク|表示|...}} → 表示

        # --- 外部リンクの平文化 ---
        (r"\[(https?://[^\s\]]+)\s+([^\]]+)\]", r"\2", 0),        # [URL ラベル] → ラベル
        (r"\[(https?://[^\s\]]+)\]", r"\1", 0),                   # [URL] → URL

        # --- 脚注の除去 ---
        (r"(?m)<ref[^>]*>.*?$", "", 0),   # 未閉じの <ref> をその行末まで削除
        (r"<ref[^>]*>.*?</ref>", "", re.S | re.I),                # <ref>…</ref> を削除
        (r"<ref[^>]*/>", "", re.I),                               # <ref name="..."/> など
        (r"<references\s*/>", "", re.I),                          # <references/>

        # --- 改行タグだけ反映 ---
        (r"<br\s*/?>", "\n", re.I),                               # <br> / <br /> → 改行

        # --- アイコン系テンプレの除去 & center の中身だけ残す---
        (r"\{\{\s*[a-z]{2,3}\s+icon\s*\}\}", "", re.I),
        (r"\{\{\s*center\|([^}]*)\}\}", r"\1", re.I),
    ]

    changed = True
    while changed:
        old = text
        for pat, repl, flg in patterns:
            text = re.sub(pat, repl, text, flags=flg | re.DOTALL)
        changed = (text != old)
    return text.strip()

info = {}
for m in re.finditer(field_pat, block, flags=re.DOTALL | re.MULTILINE):
    name = m.group(1).strip()
    value = m.group(2).strip()
    value = remove_emphasis(value)   # 26: 強調除去
    value = remove_linkmark(value)   # 27: 内部リンク平文化
    value = clean_markup_28(value)   # 28: 最小限のマークアップ除去
    info[name] = value

info

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': 'United Kingdom of Great Britain and Northern Ireland',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': 'イギリスの国章',
 '国章リンク': '（国章）',
 '標語': 'Dieu et mon droit\n（フランス語:神と我が権利）',
 '国歌': 'God Save the Queen\n神よ女王を護り賜え\nUnited States Navy Band - God Save the Queen.ogg',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '英語',
 '首都': 'ロンドン（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '女王',
 '元首等氏名': 'エリザベス2世',
 '首相等肩書': '首相',
 '首相等氏名': 'ボリス・ジョンソン',
 '他元首等肩書1': '貴族院議長',
 '他元首等氏名1': 'ノーマン・ファウラー',
 '他元首等肩書2': '庶民院議長',
 '他元首等氏名2': 'リンゼイ・ホイル',
 '他元首等肩書3': '最高裁判所長官',
 '他元首等氏名3': 'ブレンダ・ヘイル',
 '面積順位': '76',
 '面積大きさ': '1 E11',
 '面積値': '244,820',
 '水面積率': '1.3%',
 '人口統計年': '2018',
 '人口順位': '22',
 '人口大きさ': '1 E7',
 '人口値': '6643万5600',
 '人口密度値': '271',
 'GDP統計年元': '2012',
 'GDP値元': '1兆5478億',
 'GDP統計年MER': '2012',
 'GDP順位MER': '6',
 'GDP値MER': '2兆4337億',
 'GDP統計年': '2012',
 'GDP順位

In [1]:
import re

import pandas as pd


def remove_stress(dc):
    r = re.compile("'+")
    return {k: r.sub("", v) for k, v in dc.items()}


def remove_inner_links(dc):
    r = re.compile("\[\[(.+\||)(.+?)\]\]")
    return {k: r.sub(r"\2", v) for k, v in dc.items()}


def remove_mk(v):
    r1 = re.compile("'+")
    r2 = re.compile("\[\[(.+\||)(.+?)\]\]")
    r3 = re.compile("\{\{(.+\||)(.+?)\}\}")
    r4 = re.compile("<\s*?/*?\s*?br\s*?/*?\s*>")
    v = r1.sub("", v)
    v = r2.sub(r"\2", v)
    v = r3.sub(r"\2", v)
    v = r4.sub("", v)
    return v


df = pd.read_json("Jawiki Country.json.gz", lines=True)
uk_text = df.query('title=="イギリス"')["text"].values[0]
uk_texts = uk_text.split("\n")

pattern = re.compile("\|(.+?)\s=\s*(.+)")
ans = {}
for line in uk_texts:
    r = re.search(pattern, line)
    if r:
        ans[r[1]] = r[2]

r = re.compile("\[\[(.+\||)(.+?)\]\]")
ans = {k: r.sub(r"\2", remove_mk(v)) for k, v in ans.items()}
print(remove_inner_links(remove_stress(ans)))

{'略名 ': 'イギリス', '日本語国名': 'グレートブリテン及び北アイルランド連合王国', '公式国名': 'United Kingdom of Great Britain and Northern Ireland<ref>英語以外での正式国名:', '国旗画像': 'Flag of the United Kingdom.svg', '国章画像': 'イギリスの国章', '国章リンク': '（国章）', '標語': '{{lang|fr|神と我が権利）', '国歌': '[[ファイル:United States Navy Band - God Save the Queen.ogg}}', '地図画像': 'Europe-UK.svg', '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg', '公用語': '英語', '首都': 'ロンドン（事実上）', '最大都市': 'ロンドン', '元首等肩書': '女王', '元首等氏名': 'エリザベス2世', '首相等肩書': '首相', '首相等氏名': 'ボリス・ジョンソン', '他元首等肩書1': '貴族院議長', '他元首等氏名1': 'ノーマン・ファウラー', '他元首等肩書2': '庶民院議長', '他元首等氏名2': 'Lindsay Hoyle', '他元首等肩書3': '最高裁判所長官', '他元首等氏名3': 'ブレンダ・ヘイル', '面積順位': '76', '面積大きさ': '1 E11', '面積値': '244,820', '水面積率': '1.3%', '人口統計年': '2018', '人口順位': '22', '人口大きさ': '1 E7', '人口値': '6643万5600<ref>date=2019-06-26</ref>', '人口密度値': '271', 'GDP統計年元': '2012', 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=1